In [1]:
using System.IO;
using System.Collections.Generic;
using System.Collections.Concurrent;
using System.Linq;
using System.Text;
using System.Linq.Expressions;
using System.Diagnostics;

var LineSeparator = "\r\n";
var Day = "15";

public static string[] UltraSplit(this string s, string separator){
    return s.Split(separator, StringSplitOptions.RemoveEmptyEntries);
}

var inputFunction = (string day, bool isTest) => {
    var file = File.ReadAllText($"InputFiles\\D{day}{(isTest ? "_test" : "")}.txt");
    LineSeparator = file.Contains("\r\n") ? "\r\n" : "\n";
    return file.UltraSplit(LineSeparator);
};

public class Point
{
    public Point(int x, int y)
    {
        this.X = x;
        this.Y = y;
    }
    public int X;
    public int Y;

    public long TuningFrequency
    {
        get 
        {
            return (long)this.X * (long)4000000 + (long)this.Y;

        }
    }
}

public class Sensor : Point
{
    public Sensor(int x, int y, Point closestBeacon) : base(x, y)
    {
        this.ClosestBeacon = closestBeacon;
        CoverageDistance = Math.Abs(x - closestBeacon.X) + Math.Abs(y - closestBeacon.Y);
    }

    public Point ClosestBeacon;
    public int CoverageDistance;

    public (int minX, int maxX, int minY, int maxY) Boundary
    {
        get
        {
            return (this.X - CoverageDistance, this.X + CoverageDistance, this.Y - CoverageDistance, this.Y + CoverageDistance);
        }
    }

    public int[]? GetRowExclusionRange(int y)
    {
        var newDistance = Math.Abs(this.Y - y);
        if (newDistance > CoverageDistance)
        {
            return null;
        }
        else
        {
            var result = new int[2];
            result[0] = this.X - (CoverageDistance - newDistance);
            result[1] = this.X + (CoverageDistance - newDistance);
            return result;
        }
    }

    public bool IsCovered(Point beacon, bool ignoreClosestBeacon = false)
    {
        bool isCovered = false;
        if (this.ClosestBeacon.X != beacon.X || this.ClosestBeacon.Y != beacon.Y || ignoreClosestBeacon)
        {
            var newSensor = new Sensor(this.X, this.Y, beacon);
            isCovered = CoverageDistance >= newSensor.CoverageDistance;
        }
        return isCovered;
    }
}

var SolvingFunction = (string[] s) => { 
    var point = s[1].UltraSplit(",");
    var beacon = new Point(Convert.ToInt32(point[0]), Convert.ToInt32(point[1]));
    point = s[0].UltraSplit(",");
    var sensor = new Sensor(Convert.ToInt32(point[0]), Convert.ToInt32(point[1]), beacon);
    return sensor;
};

var PuzzleFunction = (bool isTest, Func<string, bool, string[]> inputFunction, Func<string[], Sensor> solvingFunction, int row, int range) => {
    var input = inputFunction(Day, isTest);
    var sensorList = new List<Sensor>();
    int minX = 0, maxX = 0, minY = 0, maxY = 0;
    foreach (var line in input)
    {
        var points = line.Replace("Sensor at x=", "")
                        .Replace(" closest beacon is at x=", "")
                        .Replace(" y=", "")
                        .UltraSplit(":");
        var sensor = solvingFunction(points);
        minX = Math.Min(sensor.X - sensor.CoverageDistance, minX);
        minX = Math.Min(sensor.ClosestBeacon.X, minX);
        minY = sensor.Y < minY ? sensor.Y : minY;
        minY = sensor.ClosestBeacon.Y < minY ? sensor.ClosestBeacon.Y : minY;

        maxX = Math.Max(sensor.X + sensor.CoverageDistance, maxX);
        maxX = Math.Max(sensor.ClosestBeacon.X, maxX);
        maxY = sensor.Y > maxY ? sensor.Y : maxY;
        maxY = sensor.ClosestBeacon.Y > maxY ? sensor.ClosestBeacon.Y : maxY;

        sensorList.Add(sensor);
    }

    // minX = minX * 3;
    // maxX = maxX * 3;

    if (range == 0)
    {
        var coveredPoints = Enumerable.Range(minX, maxX - minX).Select(x => new Point(x, row)).Count(b => sensorList.Any(s => s.IsCovered(b)));
        Console.WriteLine((isTest ? "Test" : "Puzzle") + $" Result: {coveredPoints}");
    }
    else
    {
        minX = minX < 0 ? 0 : minX;
        maxX = range < maxX ? range : maxX;
        minY = minY < 0 ? 0 : minY;
        maxY = range < maxY ? range : maxY;
        var yRange = Enumerable.Range(minY, maxY - minY);

        var points = yRange.Select(y => new Point(0, y)).ToList();

        var concurrentStack = new ConcurrentStack<Point>();
        var casioF91W = new Stopwatch();
        casioF91W.Start();
        var result = Parallel.ForEach(yRange, (y, loopState) =>
        {
            Point? distressPoint = default;

            var start = minX;
            var end = maxX;
            var exclusionRanges = sensorList.Select(s => s.GetRowExclusionRange(y)).Where(r => r != null && !(r[1] < start || r[0] > end)).ToList();
            var pointsToCheck = new List<int>();

            var count = exclusionRanges.Count;
            foreach (var er in exclusionRanges.OrderBy(er => er[0]))
            {
                if (start < er[0])
                {
                    pointsToCheck.AddRange(Enumerable.Range(start, er[0] - start));
                    start = er[1] + 1;
                }
                else if (start < er[1])
                {
                    start = er[1] + 1;
                }
                if (--count == 0)
                {
                    if (end > er[1] && end >= start)
                    {
                        pointsToCheck.AddRange(Enumerable.Range(start, end - start));
                    }
                }
            }

            distressPoint = pointsToCheck.Select(x => new Point(x, y)).FirstOrDefault(p => !sensorList.Any(s => s.IsCovered(p, true)));

            if (distressPoint != default)
            {
                concurrentStack.Push(distressPoint);
                loopState.Stop();
            }
        });

        casioF91W.Stop();
        Console.WriteLine($"ms:{casioF91W.ElapsedMilliseconds}");
        concurrentStack.TryPop(out Point distressPoint);
        Console.WriteLine((isTest ? "Test" : "Puzzle") + $" Result: x:{distressPoint.X} y:{distressPoint.Y} Frequency: {distressPoint.TuningFrequency}");
    }
};

Console.WriteLine("Problem 01:");
PuzzleFunction(true, inputFunction, SolvingFunction, 10, 0);
PuzzleFunction(false, inputFunction, SolvingFunction, 2000000, 0);

Console.WriteLine("Problem 02:");
PuzzleFunction(true, inputFunction, SolvingFunction, 10, 20);
PuzzleFunction(false, inputFunction, SolvingFunction, 10, 4000000);

Problem 01:
Test Result: 26
Puzzle Result: 5142231
Problem 02:
ms:7
Test Result: x:14 y:11 Frequency: 56000011
ms:1244
Puzzle Result: x:2721114 y:3367718 Frequency: 10884459367718
